# Classification
In this notebook we classify the CT/PET scans of lung cancer cases by tumor type. 
We are doing this as a baseline task to validate our data loading pipeline.
Most code is taken from [this tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from thermostability.thermo_dataset import ThermostabilityDataset
from util.telegram import TelegramBot

cudnn.benchmark = True

if torch.cuda.is_available():
    torch.cuda.empty_cache() 
    
cpu = torch.device("cpu")

torch.cuda.list_gpu_processes()

telegramBot = TelegramBot()
telegramBot.enabled=False

# Defining datasets (train/validation) 

In [2]:
trainSet = ThermostabilityDataset("train.csv", limit=12)
valSet = ThermostabilityDataset("train.csv", limit=12)

dataloaders = {
    "train": torch.utils.data.DataLoader(trainSet, batch_size=2, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(valSet, batch_size=2, shuffle=True, num_workers=4)
}

dataset_sizes = {"train": len(trainSet),"val": len(valSet)}
print(dataset_sizes)

{'train': 12, 'val': 12}


In [3]:
print(next(enumerate(dataloaders["train"])))

(0, [('MKDVVIQTVSANRHNYTKLAMVQFDTAAGLENITYFDDVLSFSSALNSNPPNSKNAASSTQTSDVLSVVSGFLAASGRPLESSMVVLLVNRWPAANADLTSSEYDTITNNNVKIFPISSANSYVQQIPIVTASAKIFTTLAANSNAHFVLGDYGKPLAQVFQYLMGTAYLDSLTITRSFADRTNAAQKQIGKLRVPHSETQSYVNFTITISVGLNGWGNFNYPNTNGIEVTFYQSQANQKTVQFEPGSLQGTNFYYATVQLKESSTYQVIYQSSLIDGSVAMVRVWTTSALYHYGSYATLEDPMGGNTLDKVDEYQGAALRMKLMNDCYTSHAGYAVFTDCTGAVSSKYDASQTIPIDYIFADEGSFPHYPIVPFFCDSKPKSTVDCVPGTESKYDIQFVAGEFIVNRSFQCRPGVGQINPNCTNVDSNGNYYCNRDQLPYMRGPTGQIPDCLGHGHVEYDFAFAEAYICVCDNSYSGDSCQIKN', 'MEHKGKNDFHSEWAKSIKELMLSLHEYVRQHHTTGLVWNSDPGATPMCNRKSGGAPTPPPPPPPPISLIAPSKPSGVGALLESLNTGLSATSRLKKVTPEMQTHKNPVLREVNGQMNRKTEERKVSENKKPEKIHESSIFWDGKIWKVDHQVGNKNAVVEVTDKKESIYIYKCNDSIIKIKGKANAITLDGCRKTSVVFDGLVAQCEIINCQSIQIQTLGELPTVSIQKTDGCHIYLSRDALNAQIVASKSSEMNISAMLEDGDDEYTEMALPEQFMTKIVGKKLVTVASEIV'), tensor([50.7222, 42.6682])])


# Defining model 

In [3]:
from thermostability.hotinfer import HotInferModelParallel
from thermostability.hotinfer_pregenerated import HotInferPregeneratedFC 

model = HotInferModelParallel(representation_key="esm_s_B_avg", thermo_module=HotInferPregeneratedFC(input_seq_len=2560, num_hidden_layers=3, first_hidden_size=1024))


1024
512
1024
512
A
cached_file /dhc/home/tobias.fiedler/.cache/torch/hub/checkpoints/esmfold_3B_v1.pt


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esmfold_3B_v1.pt" to /dhc/home/tobias.fiedler/.cache/torch/hub/checkpoints/esmfold_3B_v1.pt


cached_file /dhc/home/tobias.fiedler/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /dhc/home/tobias.fiedler/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt


cached_file /dhc/home/tobias.fiedler/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt


: 

: 

# Setup training

In [ ]:
from tqdm.notebook import tqdm
import sys
from util.train import train_model

# Define training parameters

In [ ]:
criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Run training

In [ ]:
try:
    model, best_epoch_loss = train_model(model, criterion, exp_lr_scheduler, dataloaders=dataloaders, dataset_sizes=dataset_sizes, use_wandb=False,
                        num_epochs=100, prepare_labels = lambda x: x.to("cuda:1"))
except Exception as e: 
    print(e)
    if not telegramBot.enabled:
        raise e
    telegramBot.send_telegram(f"Training failed with error message: {str(e)}")                         

Epoch 0/99
----------
esm_s A shape torch.Size([1, 148, 37, 2560])
esm_s B shape torch.Size([1, 148, 2560])
s_s_0 A shape torch.Size([1, 148, 1024])
s_s_0 B shape torch.Size([1, 148, 1024])
esm_s A shape torch.Size([1, 371, 37, 2560])
esm_s B shape torch.Size([1, 371, 2560])
s_s_0 A shape torch.Size([1, 371, 1024])
s_s_0 B shape torch.Size([1, 371, 1024])
Given normalized_shape=[716800], expected input with shape [*, 716800], but got input of size[2, 1792000]


RuntimeError: Given normalized_shape=[716800], expected input with shape [*, 716800], but got input of size[2, 1792000]

# Evaluation

In [9]:
from datetime import datetime

def predictDiffs(set="val"):
    with torch.no_grad():
        n = len(dataloaders[set])
        diffs = torch.tensor([])
        for index, (inputs, labels) in enumerate(dataloaders[set]):
            #inputs = inputs.to(device)
            print(f"Infering thermostability for sample {index}/{n}...")
            labels = labels.to("cuda:1")
            outputs = model(inputs)

            _diffs = outputs.squeeze().sub(labels.squeeze()).cpu()
            diffs = torch.cat((diffs, _diffs))
            print("Diff: ", _diffs)
    return diffs
diffs = predictDiffs()

#diffs = np.array([0, 0.1, 0.2,-0.2, -0.8, 0.1])
plt.title("Differences predicted <-> actual thermostability")
plt.hist(diffs, 10)
resultsDir = "results"
now = datetime.now()
time = now.strftime("%d/%m/%Y_%H:%M:%S")
os.makedirs(resultsDir, exist_ok=True)
histFile = f"results/{time}_diffs.png"
plt.savefig(histFile)
telegramBot.send_photo(histFile, f"Differences predicted <-> actual thermostability at {time}")


Infering thermostability for sample 0/6...


KeyboardInterrupt: 

# Save model

In [ ]:
try: 
    modelPath = os.path.join(resultsDir, f"{time}_model.pth")
    torch.save(model, modelPath)
    telegramBot.send_telegram(f"Model saved at {modelPath}")
except Exception as e:
    telegramBot.send_telegram(f"Saving model failed for reason: {str(e)}")